# first check that synthetically made image data given shape  2525 x 2525 x 3

# this notebook combined DALI data pipeline + model training in one go start with smaller image size 224x224 x3
## go to launch and type in **watch -n 1 nvidia-smi** in order to monitor the GPU utils as I showcased in the demo

In [1]:
### reference url :https://docs.nvidia.com/deeplearning/dali/user-guide/docs/examples/tensorflow/tensorflow-dataset.html 
### https://github.com/NVIDIA/DALI/blob/master/docs/examples/frameworks/tensorflow/tensorflow-dataset.ipynb

In [2]:
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
tf.debugging.set_log_device_placement(True)
c=random.sample(os.listdir('/workspace/small'),1)
random_img=random.sample(os.listdir('/workspace/small/'+c[0]),1)
test=plt.imread('/workspace/small/'+c[0]+'/'+random_img[0])
print(test.shape, np.min(test), np.max(test))

(224, 224, 3) 0.011764706 0.99215686


In [3]:
## make label so it is easier to visualize and do plotting 
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
label2num={'notHappy':0,'Happy':1,'others':2}
num2label=dict([(b,a) for (a,b) in label2num.items()])
device_id=0


In [4]:
!tail smallimgs_path.txt

/workspace/small/others/others_90.png 2
/workspace/small/others/others_91.png 2
/workspace/small/others/others_92.png 2
/workspace/small/others/others_93.png 2
/workspace/small/others/others_94.png 2
/workspace/small/others/others_95.png 2
/workspace/small/others/others_96.png 2
/workspace/small/others/others_97.png 2
/workspace/small/others/others_98.png 2
/workspace/small/others/others_99.png 2


---------------------------------------------------------------------------------------------------------------

# === next we add flip+ **rotate** + resize to 224x224x3  and create a new pipeline named Opt3Pipeline ===
## run through notebook = **resize2big_image.ipynb** to create corresponding director file= **bigimgs_path.txt**

In [5]:
import tensorflow as tf

import nvidia.dali.ops as ops
import nvidia.dali.types as types
from nvidia.dali.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


class Opt3Pipeline(Pipeline):
    def __init__(self, img_file_list,  num_shards, shard_id, batch_size, num_threads, device_id):
        super(Opt3Pipeline, self).__init__(batch_size, num_threads, device_id, seed = 12)
        self.input_img = ops.FileReader(file_root = "", num_shards=num_shards, shard_id=shard_id, file_list=img_file_list,random_shuffle = True, initial_fill = 21)
        self.decode = ops.ImageDecoder(device = "mixed", output_type = types.RGB)
       


    def define_graph(self):
        imgs, labels = self.input_img()
        images = self.decode(imgs)
        return (images, labels)
"""
batch_size=2
pipe3 = Opt3Pipeline("smallimgs_path.txt", 2, 0, batch_size, 8, 0)

pipe3.build()

images3, labels3 = pipe3.run()
len_outputs = 8

imgs3=np.array(images3.as_cpu().as_tensor())
print(imgs3.shape)
mas3=np.array(labels3.as_tensor())
print(type(imgs3),type(mas3))
print(imgs3.shape, mas3.shape)"""

'\nbatch_size=2\npipe3 = Opt3Pipeline("smallimgs_path.txt", 2, 0, batch_size, 8, 0)\n\npipe3.build()\n\nimages3, labels3 = pipe3.run()\nlen_outputs = 8\n\nimgs3=np.array(images3.as_cpu().as_tensor())\nprint(imgs3.shape)\nmas3=np.array(labels3.as_tensor())\nprint(type(imgs3),type(mas3))\nprint(imgs3.shape, mas3.shape)'

In [6]:
## sanity check the version of dali and tf

In [7]:
import nvidia.dali as dali
print(dali.__version__)
print(tf.__version__)

0.16.0
2.0.0


In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow.compat.v1 as tf
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.plugin.tf as dali_tf
tf.logging.set_verbosity(tf.logging.ERROR)
batch_size=8

def get_reset50_model():
    # load model without classifier layers
    num_class=3
    base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))

    # make all layers trainable
    for layer in base_model.layers:
        layer.trainable = True
    # add your head on top
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    output = Dense(num_class, activation='softmax')(x)

    m = Model(inputs=base_model.input, outputs=output)
    return m

pipe3 = Opt3Pipeline("smallimgs_path.txt", 2, 0, batch_size, 8, 0)
with tf.device('/gpu:0'):
    

    # Create dataset
    # Define shapes and types of the outputs
    shapes = [
        (batch_size, 224, 224,3),
        (batch_size,1)]
    dtypes = [
        tf.float32,
        tf.int32]
    out = dali_tf.DALIDataset(
            pipeline=pipe3,
            batch_size=batch_size,
            shapes=shapes,
            dtypes=dtypes,
            device_id=0)

    
    #out = out.with_options(dataset_options())

    m=get_reset50_model()
    m.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    # Train using DALI dataset
    m.fit(
        out,
        epochs=50,
        steps_per_epoch=batch_size*8,
        use_multiprocessing=False
    )


Executing op DALIDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas